In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter

import datetime

#### **주어진 학습용 데이터(used_car_y_train.csv, used_car_x_train.csv)를 이용하여 중고차의 판매 가격을 예측하고, 이를 평가용 데이터(used_car_x_test.csv)에 적용하여 얻은 예측 결과를 다음과 같은 형식의 csv 파일로 생성하시오.** 

In [4]:
x_test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/used_car_X_test.csv')
x_train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/used_car_X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/used_car_y_train.csv')

print(x_train.head())

   id model  year transmission  mileage fuelType  tax   mpg  engineSize
0   1    A1  2019    Automatic     3500   Petrol  145  40.9         2.0
1   2   RS4  2020    Semi-Auto     2500   Petrol  145  28.8         2.9
2   3    A8  2019    Semi-Auto      500   Diesel  145  40.4         3.0
3   4    Q5  2019    Semi-Auto     5089   Diesel  150  38.2         2.0
4   5    A5  2020    Semi-Auto     4951   Diesel  145  51.4         2.0


In [6]:
# 결측치 확인 
print(x_test.isnull().sum().sum())
print(x_train.isnull().sum().sum())
print(y_train.isnull().sum().sum())

0
0
0


In [11]:
x_train.head()

,id,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,1,A1,2019,Automatic,3500,Petrol,145,40.9,2.0
1,2,RS4,2020,Semi-Auto,2500,Petrol,145,28.8,2.9
2,3,A8,2019,Semi-Auto,500,Diesel,145,40.4,3.0
3,4,Q5,2019,Semi-Auto,5089,Diesel,150,38.2,2.0
4,5,A5,2020,Semi-Auto,4951,Diesel,145,51.4,2.0


In [16]:
# 데이터 나누기
# 범주형 데이터
x_train_word = x_train[['model', 'transmission', 'fuelType']]
x_test_word = x_test[['model', 'transmission', 'fuelType']]

# 수치형 데이터
x_train_num = x_train.drop(columns = ['model', 'transmission', 'fuelType'])
x_test_num = x_test.drop(columns = ['model', 'transmission', 'fuelType'])

x_test_num.head()

,id,year,mileage,tax,mpg,engineSize
0,7469,2020,4000,145,54.3,3.0
1,7470,2016,31300,30,61.4,2.0
2,7471,2019,11296,150,38.2,2.0
3,7472,2018,6500,145,55.4,1.4
4,7473,2016,31524,30,60.1,1.4


# **from sklearn.preprocessing import MinMaxScaler**
### **.fit_transform()**

#### **MinMaxScaler: 데이터 정규화 방법 중 하나로, 데이터의 값을 0과 1사이의 범위로 변환**
#### **데이터의 최소값과 최대값을 기준으로 계산**

In [21]:
# 데이터 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

x_train_num_scale = scaler.fit_transform(x_train_num)
x_test_num_scale = scaler.fit_transform(x_test_num)
 
# 데이터 프레임 설정 (array 형태임)
df_train_num = pd.DataFrame(x_train_num_scale, columns=x_train_num.columns)
df_test_num = pd.DataFrame(x_test_num_scale, columns=x_test_num.columns)

# 원핫 인코딩
# 범주형 데이터를 이진 벡터로 변환
df_train_word = pd.get_dummies(x_train_word)
df_test_word =pd.get_dummies(x_test_word)

### **set(): 자동으로 중복된 요소 제거, 집합 자료형 생성**

In [26]:
# 훈련 데이터프레임 컬럼 목록
train_columns = set(df_train_word.columns)

# 테스트 데이터프레임 컬럼 목록
test_columns = set(df_test_word.columns)

# 훈련 데이터에는 있지만 테스트 데이터에는 없는 컬럼 확인
missing_in_test = train_columns - test_columns
print("missing_in_test: ", missing_in_test)

# 테스트 데이터에는 있지만 훈련 데이터에는 없는 컬럼 확인
missing_in_train = test_columns - train_columns
print("missing_in_train: ", missing_in_train)

# 컬럼 목록 중 없는 컬럼의 값을 0으로 해서 채워줌
df_test_word['model_S8'] = 0
df_test_word['model_ RS7'] = 0
df_test_word['model_ A2'] = 0
df_test_word['model_ S5'] = 0

# 데이터 결합 (이진형 데이터 + 범주형 데이터의 결합)
# merge(df1, df2, on='key', how='outer'): 공통 열(키) 또는 인덱스를 기준으로 데이터 병합
# concat([df1, df2], axis=1): 축(axis)을 따라 연결
df_train = pd.concat([df_train_num, df_train_word], axis=1)
df_test = pd.concat([df_test_num, df_test_word], axis=1)
print()
print(df_train)

missing_in_test:  {'model_ S8'}
missing_in_train:  {'model_S8'}

            id      year   mileage       tax       mpg  engineSize  model_ A1  \
0     0.000000  0.954545  0.020825  0.250000  0.127811    0.317460       True   
1     0.000134  1.000000  0.014874  0.250000  0.056213    0.460317      False   
2     0.000268  0.954545  0.002970  0.250000  0.124852    0.476190      False   
3     0.000402  0.954545  0.030283  0.258621  0.111834    0.317460      False   
4     0.000536  1.000000  0.029461  0.250000  0.189941    0.317460      False   
...        ...       ...       ...       ...       ...         ...        ...   
7463  0.999464  0.818182  0.134701  0.051724  0.234320    0.222222      False   
7464  0.999598  0.863636  0.078987  0.250000  0.265680    0.253968      False   
7465  0.999732  0.954545  0.029753  0.250000  0.127811    0.317460       True   
7466  0.999866  0.954545  0.000054  0.250000  0.139053    0.317460      False   
7467  1.000000  0.636364  0.542972  0.517241

In [27]:
y_train.head()

,id,price
0,1,21350
1,2,69691
2,3,42950
3,4,31470
4,5,27495


# **from pandas.core.common import random_state**
## **import xgboost as xgb**
## **from sklearn.metrics import mean_squared_error**

In [33]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [35]:
# 모델 학습
from pandas.core.common import random_state
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# 모델 생성
model = xgb.XGBRegressor(random_state=77)

# train , validation 데이터 설정
X_train, X_val, y_train_t, y_val = train_test_split(df_train.values, 
                                                    y_train['price'].values, 
                                                    test_size=0.3)

# 모델 학습
model.fit(X_train, y_train_t)

# vaidation 데이터로 성능 평가
y_pred = model.predict(X_val)
print(np.sqrt(mean_squared_error(y_val, y_pred)))

2356.7913501314856


#### fit_transform()은 변환기(Transformer) 객체에서 사용되는 메서드이고, fit()은 모델 학습을 위한 메서드
#### **fit_transform()은 변환기(Transformer)에서 주로 사용됩니다. 예를 들어, StandardScaler, MinMaxScaler, PCA와 같은 변환기에서 데이터의 스케일링, 차원 축소 등을 할 때 사용**

In [37]:
# 예측
y_pred = model.predict(df_test)

df = pd.DataFrame(x_test['id'], columns=['id'])
# 예측값 컬럼 추가
df['price'] = y_pred
print(df.head())

     id         price
0  7469  32254.832031
1  7470  21927.570312
2  7471  30821.490234
3  7472  13200.222656
4  7473  18354.972656
